In [ ]:
from compare_adversarial import *

2024-06-12 12:17:27.329851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
cnn = load_model("trained_models/CNN/classifier.h5")
# input shape: (28, 28, 1)

fgsm = FGSM(cnn)
dfuz = DLFuzz(cnn)

vae = VAE.load("trained_models/VAE")
lava = Lava(cnn, vae.encoder, vae.decoder)
lavam = LavaMultiSteps(cnn, vae.encoder, vae.decoder, verbose=True)

# input images
samples, sample_labels, sample_indices = load_samples_for_test(200, return_indices=True)


y_onehot = tf.one_hot(tf.constant(sample_labels), depth=10).numpy()

ValueError: bad marshal data (unknown type code)

In [4]:
results = []
# Start experiment
for i in tqdm(range(len(samples))):
    # Get original image and label
    image_org, label_org = samples[i], sample_labels[i]

    # Generate adversarial images of shape (28, 28, 1)
    adversarial_images = {
        "fgsm": fgsm.generate_adversarial_image(image_org, y_onehot[i]),
        "dlfuzz": dfuz.generate_adversarial_image(image_org),
        "lava": lava.generate_adversarial_image(image_org, y_onehot[i])[0],
        "lavam": lavam.generate_adversarial_image(image_org, y_onehot[i], alpha=0.05, num_iterations=10)[0],
    }
    # adversarial_images["lava"], h_lava = lava.generate_adversarial_image(image_org, y_onehot[i])

    # Predict labels for adversarial images
    predicted_labels = {
        key: np.argmax(cnn.predict(np.array([img]))[0])
        for key, img in adversarial_images.items()
    }

    # Plot the images if original label differs from all adversarial labels
    if label_org not in predicted_labels.values():
        images = [image_org] + list(adversarial_images.values())
        labels = [label_org] + list(predicted_labels.values())
        markers = ["org"] + list(adversarial_images.keys())
        plot_image_comparison(images, labels, markers)

        # In image space
        divergence_functions = {
            "kl": kl_divergence,
            "ws": ws_distance,
            "js": js_divergence,
            # "mse": mse_loss,
            # "xentropy", cross_entropy,
        }
        result = dict()
        result["index"] = i
        result["index_in_dataset"] = sample_indices[i]
        for name, func in divergence_functions.items():
            divergences = {key: func(image_org, img) for key, img in adversarial_images.items()}
            for key, value in divergences.items():
                result[f"{name}_{key}"] = value
                print(f"{name}_{key}: {value}")
            print("-"*64)

        results.append(result)

  0%|          | 0/200 [00:00<?, ?it/s]

2024-06-12 12:11:04.395545: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:675 : INVALID_ARGUMENT: Computed output size would be negative: -1 [input_size: 1, effective_filter_size: 3, stride: 1]


InvalidArgumentError: Exception encountered when calling layer "conv2d" (type Conv2D).

Computed output size would be negative: -1 [input_size: 1, effective_filter_size: 3, stride: 1] [Op:Conv2D]

Call arguments received by layer "conv2d" (type Conv2D):
  • inputs=tf.Tensor(shape=(28, 28, 1, 1), dtype=float32)